# <font color="orange">参考サイト</font>
https://baseballsavant.mlb.com/statcast_search (トラックマンデータを取得)    
https://yakyumaru.com/2021/01/21/post-268/    （列名の説明を参考にした）   
https://shinyorke.hatenablog.com/entry/statcast-csv-docs-ja    (ストライクゾーンの区画を参考にした)    
https://ja.wikipedia.org/wiki/%E6%9C%80%E5%A4%9A%E6%9C%AC%E5%A1%81%E6%89%93_(MLB)　（シーズン別のホームラン王を参考にした）     
https://bootswatch.com/simplex/   （dbc.themesの詳細）


# <font color="orange">備考</font>
大谷翔平のメジャーリーグでのバッティングデータ(2018/4/1 ~ 2023/8/23)   
トラックマンデータよりデータを入手。   
使用の列のみ抽出した。    
2020年はリハビリしながらのマイナーリーグでの出場のため、試合数が少ない。   
2021年はMVP受賞。

# <font color="orange">インポート文</font>

In [1]:
# import文
import numpy as np
import pandas as pd
from IPython.display import display

# <font color="orange">csvファイルの読み込み</font>

### <font color="turquoise">バッター大谷のデータ読み込み</font>

In [2]:
# マイルをkm/hに変換し、フィートをcmに変換する。
data_batter = pd.read_csv("./data/Ohtani_all_season_batter.csv")
data_batter.release_speed = data_batter.release_speed * 1.61    # マイルをkm/hに変換。
data_batter.release_pos_x = data_batter.release_pos_x * 30.48   # フィートをcmに変換。
data_batter.release_pos_z = data_batter.release_pos_z * 30.48   # フィートをcmに変換。
data_batter.plate_x = data_batter.plate_x * 30.48   # フィートをcmに変換。
data_batter.plate_z = data_batter.plate_z * 30.48   # フィートをcmに変換。
data_batter.sz_top = data_batter.sz_top * 30.48   # フィートをcmに変換。
data_batter.sz_bot = data_batter.sz_bot * 30.48   # フィートをcmに変換。

# 使用するカラムだけを抽出。
data_batter = data_batter[["game_year",
                           "game_date",
                           "p_throws",       # 左投げ投手 or 右投げ投手
                           "pitch_name",    # 球種
                           "events",        # 結果（アウト、シングルヒット、デッドボールなど）
                           "description",   # 一級ごとの詳細（ファール、空振りなど）
                           "zone",          # コースを13分割にしたもの。詳細はURL参照。
                           "plate_x",       # ホームプレート上での水平方向の通過点。
                           "plate_z",       # ホームプレート上での鉛直方向の通過点。
                           "launch_speed",  # 打球の初速。
                           "launch_angle"   # 打球の軌道。
             ]]
data_batter.dropna(subset=["zone"], inplace=True)
data_batter.reset_index(drop=True, inplace=True)

### <font color="turquoise">ピッチャー大谷のデータ読み込み</font>

In [3]:
# マイルをkm/hに変換し、フィートをcmに変換する。
data_pitcher = pd.read_csv("./data/Ohtani_all_season_pitch.csv")
data_pitcher.release_speed = data_pitcher.release_speed * 1.61    # マイルをkm/hに変換。
data_pitcher.release_pos_x = data_pitcher.release_pos_x * 30.48   # フィートをcmに変換。
data_pitcher.release_pos_z = data_pitcher.release_pos_z * 30.48   # フィートをcmに変換。
data_pitcher.plate_x = data_pitcher.plate_x * 30.48    # フィートをcmに変換。
data_pitcher.plate_z = data_pitcher.plate_z * 30.48    # フィートをcmに変換。
data_pitcher.sz_top = data_pitcher.sz_top * 30.48    # フィートをcmに変換。
data_pitcher.sz_bot = data_pitcher.sz_bot * 30.48    # フィートをcmに変換。

# 使用するカラムだけを抽出。
data_pitcher = data_pitcher[["game_year",
                             "game_date",
                             "stand",         # 左打者 or 右打者
                             "release_speed", # 球速
                             "pitch_name",    # 球種
                             "events",        # 結果（アウト、シングルヒット、デッドボールなど）
                             "description",   # 一級ごとの詳細（ファール、空振りなど）
                             "zone",          # コースを13分割にしたもの。詳細はURL参照。
                             "plate_x",       # ホームプレート上での水平方向の通過点。
                             "plate_z"        # ホームプレート上での鉛直方向の通過点。
             ]]
data_pitcher.dropna(subset=["zone"], inplace=True)
data_pitcher.dropna(subset=["pitch_name"], inplace=True)
data_pitcher.reset_index(drop=True, inplace=True)


# <font color="orange">関数集</font>

In [4]:
# データフレームを水平に表示する。
def show_many_dfs(*dfs, n=10):
    class HorizontalDisplay:
        def _repr_html_(self):
            template = '<div style="float: left; padding: 5px;">{}</div>'
            return  ''.join(template.format(df.head(n)._repr_html_()) for df in dfs)
    return HorizontalDisplay()

In [5]:
# zoneごとのデータフレームを作成する（ヒートマップを出力するためのデータフレーム）。
def make_df_zone_count(df):
    
    # 引き取ったデータフレームから"zone"列を抽出し、コースごとに投げられた回数をカウントしたデータフレームを作成する。
    df_zone = pd.DataFrame(df.zone.value_counts())
    df_zone = df_zone.sort_values(by = "zone").reset_index()
    df_zone.columns = ["zone", "count"]
    df_zone.zone = df_zone.zone.astype("int")
    
    # 引き取ったデータフレームによっては、一度も投げられていないコースがある可能性の対策として、
    # zoneフレームをあらかじめ作成しておき、mergeする。
    df_zone_value = pd.DataFrame([1,2,3,4,5,6,7,8,9,11,12,13,14])    # 1~9はストライクゾーン。11~14はボールゾーン。
    df_zone_value.columns = ["zone"]

    df_zone_count = pd.merge(df_zone_value, df_zone, on = "zone", how = "left").fillna(0)
    df_zone_count.set_index("zone", inplace=True)
    
    # df_zone_countの形を整える
    z01 = df_zone_count.query("zone == 1").loc[1, "count"]
    z02 = df_zone_count.query("zone == 2").loc[2, "count"]
    z03 = df_zone_count.query("zone == 3").loc[3, "count"]
    z04 = df_zone_count.query("zone == 4").loc[4, "count"]
    z05 = df_zone_count.query("zone == 5").loc[5, "count"]
    z06 = df_zone_count.query("zone == 6").loc[6, "count"]
    z07 = df_zone_count.query("zone == 7").loc[7, "count"]
    z08 = df_zone_count.query("zone == 8").loc[8, "count"]
    z09 = df_zone_count.query("zone == 9").loc[9, "count"]
    z11 = df_zone_count.query("zone == 11").loc[11, "count"]
    z12 = df_zone_count.query("zone == 12").loc[12, "count"]
    z13 = df_zone_count.query("zone == 13").loc[13, "count"]
    z14 = df_zone_count.query("zone == 14").loc[14, "count"]
    
    df_zone_count = pd.DataFrame([[z11, z11, z11, z11, z11, z12, z12, z12, z12, z12],
                                  [z11, z11, z11, z11, z11, z12, z12, z12, z12, z12],
                                  [z11, z11, z01, z01, z02, z02, z03, z03, z12, z12],
                                  [z11, z11, z01, z01, z02, z02, z03, z03, z12, z12],
                                  [z11, z11, z04, z04, z05, z05, z06, z06, z12, z12],
                                  [z13, z13, z04, z04, z05, z05, z06, z06, z14, z14],
                                  [z13, z13, z07, z07, z08, z08, z09, z09, z14, z14],
                                  [z13, z13, z07, z07, z08, z08, z09, z09, z14, z14],
                                  [z13, z13, z13, z13, z13, z14, z14, z14, z14, z14],
                                  [z13, z13, z13, z13, z13, z14, z14, z14, z14, z14],
                                  ])
    return df_zone_count

# <font color="orange">列名の説明（不明が複数あり。必要になったら調べる）</font>

In [6]:
# 列名の説明csvファイルを読み込み、データフレームを作成
df_columns_explanation = pd.read_csv("./data/trackmandata_columns_explanation.csv")

pd.options.display.max_rows=100
df_columns_explanation[~df_columns_explanation["説明"].str.contains('不明')].head(100)

,列名,説明
0,pitch_type,球種（pitch_nameと同じ）
1,game_date,日付
2,release_speed,球速（マイル表示。 ×1.61でkm/hに変換）
3,release_pos_x,水平リリース位置（フィート表示。 ×30.48でcmに変換）
4,release_pos_z,垂直リリース位置（フィート表示。 ×30.48でcmに変換）
6,batter,打者の管理番号
7,pitcher,投手の管理番号（同じ列が二つある）
8,events,結果（アウト、シングルヒット、デッドボールなど）
9,description,一球ごとの結果
14,zone,投球コース（1〜9はストライクゾーン、11〜14はボールゾーン）


# <font color="orange">dashアプリケーションの定義</font>

## <font color="tomato">ダッシュボード用インポート文</font>

In [7]:
# import分
import dash
import dash_bootstrap_components as dbc
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objs as go

## <font color="tomato">dashアプリケーションの初期化</font>

In [8]:
app = dash.Dash(external_stylesheets=[dbc.themes.SIMPLEX])

## <font color="tomato">表示させるデータのデータフレーム、レイアウトの作成</font>

### <font color="turquoise">サイドバーの作成</font>

In [9]:
sidebar = html.Div(
    [
        html.Div(
            [
                html.H2('Settings',
                        style={"width": "100%",
                               'font-family': 'Copperplate'
                        }
                ),
            ],
            className='bg-primary text-white text-center'
        ),
        html.Div(
            [
                html.H5('カテゴリ選択',
                        className='bg-light text-primary text-center'
                ),                
                dcc.RadioItems(
                    id='表示させるデータ_セレクトボタン',
                    value="ホーム",
                    options=[
                        {'label': 'ホーム', 'value': 'ホーム'},
                        {'label': '基本データ', 'value': '基本データ'},
                        {'label': 'MLB試合出場数', 'value': 'MLB試合出場数'},
                        {'label': 'バッティングデータ', 'value': 'バッティングデータ'},
                        {'label': 'ピッチングデータ', 'value': 'ピッチングデータ'}
                    ],
                    style={"width": "100%"}
                ),
            ],
            className='bg-primary text-white mb-4'
        ),
        html.Div(
            [
                html.H5('データの選択',
                        id = 'データの種類_テキスト',
                        className='bg-light text-primary text-center'
                ),
                dcc.RadioItems(id='データの種類_セレクトボタン')
            ],
            className='bg-primary text-white'
        )
    ]
)
# コールバック関数を定義
@app.callback(
    Output('データの種類_テキスト', 'style'),
    Output('データの種類_セレクトボタン', 'options'),
    Output('データの種類_セレクトボタン', 'value'),
    Output('データの種類_セレクトボタン', 'style'),
    Input('表示させるデータ_セレクトボタン', 'value')
)
def update_options_sidebar(selected_data):
    
    # データの選択ボタンを表示させないカテゴリ以外用に、空のスタイルをreturnする。
    default_style_text = {}
    default_style_options = {}
    
    if selected_data == 'ホーム':
        return {'display': 'none'}, [], None, {'display': 'none'} 
               
    elif selected_data == '基本データ':
        options = [{'label': '基本情報', 'value': '基本情報'},
                   {'label': '経歴', 'value': '経歴'},
                   {'label': '獲得タイトル', 'value': '獲得タイトル'},
                   {'label': 'その他の逸話', 'value': 'その他の逸話'}]
        return default_style_text, options, options[0]["value"], default_style_options
    
    elif selected_data == 'MLB試合出場数':
        return {'display': 'none'}, [], None, {'display': 'none'} 
               
    elif selected_data == 'バッティングデータ':
        options = [{'label': '年間別ホームラン数', 'value': '年間別ホームラン数'},
                   {'label': '投げられたコース', 'value': '投げられたコース'},
                   {'label': '投げられた球種', 'value': '投げられた球種'}]        
        return default_style_text, options, options[0]["value"], default_style_options
               
    elif selected_data == 'ピッチングデータ':
        options = [{'label': '持ち玉と配球', 'value': '持ち玉と配球'},
                   {'label': '球種ごとのデータ', 'value': '球種ごとのデータ'},
                   {'label': 'ピッチングデータ3', 'value': 'ピッチングデータ3'}]
        return default_style_text, options, options[0]["value"], default_style_options

### <font color="turquoise">ホームのレイアウト作成</font>

In [10]:
# ホーム画面に表示させる画像をencodeする。
import base64

# 画像ファイルを指定
image_filename = "img/home_img.jpg"

# 画像ファイルをバイナリモードで開いて読み込み
with open(image_filename, 'rb') as f:
    # 画像ファイルをBase64エンコード
    encoded_image = base64.b64encode(f.read()).decode('ascii')


layout_home = dbc.Container(
    [
        dbc.Row(
            [
                html.H1("大谷翔平選手のメジャーリーグの活躍について",
                        className='text-light')
            ]
        ),
        dbc.Row(
            [
                html.Div(
                    [
                        html.Img(src=f"data:image/jpeg;base64,{encoded_image}",
                                 alt="画像の表示に失敗しました",
                        # html.Img(src="https://source.unsplash.com/random",
                                 style={"height": "90vh",
                                        "width" : "auto"},
                                 className="text-light")
                    ],
                    style={"text-align": "center"}
                )
            ]
        ),
        dbc.Row(
            [
                html.Small("※本ダッシュボードは、大谷翔平選手のMLB移籍後から2023年度までのデータを用いて作成しています。",
                            style={"text-align": "right"},
                            className="text-light")
            ]
        )
    ],
    # fluid=True   # 意図しない余白が出ないように設定。
)

### <font color="turquoise">基本データのレイアウト作成</font>

In [11]:
# 基本データのレイアウト

margin_bottom = "30px"
layout_basic_information = dbc.Container(
    [
        html.Div(
            [
                dbc.Row(
                    [
                        html.H1("大谷翔平選手の基本情報",
                                style={"margin-bottom" : "50px"})
                    ],
                    className="text-light"
                ),
                dbc.Row(
                    [
                        html.H2("・生年月日 : 1994年7月5日",
                                style={"margin-bottom" : margin_bottom}),

                        html.H2("・出身地 : 岩手県水沢市（現 : 奥州市）",
                                style={"margin-bottom" : margin_bottom}),

                        html.H2("・血液型 : B型",
                                style={"margin-bottom" : margin_bottom}),

                        html.H2("・身長 : 約193cm",
                                style={"margin-bottom" : margin_bottom}),

                        html.H2("・体重 : 約95kg",
                                style={"margin-bottom" : margin_bottom}),

                        html.H2("・投球・打席 : 右投げ左打ち",
                                style={"margin-bottom" : margin_bottom}),
                    ],
                    className="text-light"
                )
            ]
        )
    ],
    # fluid=True
)


layout_career = dbc.Container(
    [
        html.Div(
            [
                dbc.Row(
                    [
                        html.H1("大谷翔平選手の経歴",
                                style={"margin-bottom" : "50px"}),
                    ],
                    className="text-light"
                ),
                dbc.Row(
                    [                    
                        html.H2("・2002年（当時小学2年生）: 友達からの誘いがきっかけで、水沢リトルリーグで野球を始める",
                               style={"margin-bottom" : margin_bottom}),

                        html.H2("・2010-2012年 : 花巻東高校（岩手県）に進学し、2度の甲子園出場を果たす（どちらも一回戦敗退）",
                               style={"margin-bottom" : margin_bottom}),

                        html.H2("・2012年 : 日本ハムより一位指名で入団。二刀流選手として試合出場。",
                               style={"margin-bottom" : margin_bottom}),

                        html.H2("・2017年 : ロサンゼルス・エンゼルスと契約を合意",
                               style={"margin-bottom" : margin_bottom}),

                        html.H2("・2018年 : タイトル '新人王' 獲得",
                               style={"margin-bottom" : margin_bottom}),

                        html.H2("・2021年 : シーズンMVP",
                               style={"margin-bottom" : margin_bottom}),

                        html.H2("・2023年 : 最多本塁打王",
                               style={"margin-bottom" : margin_bottom})
                    ],
                    className="text-light"
                )
            ]
        )
    ],
    # fluid=True
)


layout_title = dbc.Container(
    [
        html.Div(
            [
                dbc.Row(
                    [
                        html.H1("大谷翔平選手の獲得タイトル",
                                style={"margin-bottom" : "50px"}),
                    ],
                    className="text-light"
                ),
                dbc.Row(
                    [                    
                        html.H2("・最多勝利(2015年)",
                               style={"margin-bottom" : margin_bottom}),

                        html.H2("・最優秀防御率(2015年)",
                               style={"margin-bottom" : margin_bottom}),

                        html.H2("・最優秀勝率(2015年)",
                               style={"margin-bottom" : margin_bottom}),

                        html.H2("・ベストナイン投手(2015, 16年)",
                               style={"margin-bottom" : margin_bottom}),

                        html.H2("・MLB新人王(2018年)",
                               style={"margin-bottom" : margin_bottom}),

                        html.H2("・シーズンMVP(2021年)",
                               style={"margin-bottom" : margin_bottom}),

                        html.H2("・最多本塁打王(2023年)",
                               style={"margin-bottom" : margin_bottom}),
                    ],
                    className="text-light"
                )
            ]
        )
    ],
    # fluid=True
)

margin_bottom_for_layout_others = "15px"
layout_others = dbc.Container(
    [
        html.Div(
            [
                dbc.Row(
                    [
                        html.H1("大谷翔平選手のその他の逸話",
                                style={"margin-bottom" : "50px"}),
                    ],
                    className="text-light"
                ),
                dbc.Row(
                    [
                        html.H2("〜中学時代",
                                style={"margin-bottom" : margin_bottom_for_layout_others}),             
                        html.H3("　・小学5年生のときに球速110km/hを記録。"),
                        html.H3("　・中学1年生のときの福島県合宿で、ホームランが場外となり、信号機を壊す。（推定飛距離120m）"),
                        html.H3("　　そのため、ピッチャーライナーを打つことを禁止されていた。")
                    ],
                    class_name="mb-4"
                ),
                dbc.Row(
                    [           
                        html.H2("高校時代",
                                style={"margin-bottom" : margin_bottom_for_layout_others}),
                        html.H3("　・高校1年生で、球速147km/h"),

                        html.H3("　・高校2年生の夏の甲子園で、球速151km/h（田中将大投手がマークした甲子園最速記録タイ）"),

                        html.H3("　・高校3年生の春の甲子園で、球速160km/h（アマチュア史上初の記録）"),

                        html.H3("　・高校通算ホームラン数56本（歴代25位）"),
                    ],
                    class_name="mb-4"
                ),
                dbc.Row(
                    [           
                        html.H2("日本ハム時代",
                                style={"margin-bottom" : margin_bottom_for_layout_others}),    
                        html.H3("　・2014年（当時20歳）で16奪三振の最年少記録を達成"),

                        html.H3("　・2016年（当時22歳）で日本最速の球速165km/hを記録"),
                    ],
                    class_name="mb-4"
                ),
                dbc.Row(
                    [        
                        html.H2("ロサンゼルス・エンゼルス時代",
                                style={"margin-bottom" : margin_bottom_for_layout_others}),          
                        html.H3("　・ベストMLBプレイヤー賞を3年連続で受賞（2021-2023年）"),

                        html.H3("　・審査員30人全員が大谷翔平に投票し、満場一致でMVPを受賞（2021年）"),

                        html.H3("　・MLB史上初となる2年連続の「2桁勝利&2桁本塁打」を達成（2022, 23年）"),

                        html.H3("　・MLB史上初となる2部門（投手部門・指名打者部門）でのオールWBCチームに選出され、MVPを受賞（2023年）"),
                    ]
                )
            ],
            className="text-light"
        )
    ],
    # fluid=True
)

### <font color="turquoise">MLB試合出場数のデータフレーム・レイアウト作成</font>

##### <font color="plum">データフレームの作成</font>

In [12]:
# 最終試合出場年を取得する。
if data_batter.game_year.max() > data_pitcher.game_year.max():
    last_game_year = data_batter.game_year.max()
else:
    last_game_year = data_pitcher.game_year.max()

# デフォルトの年毎のデータフレームを作成する。
# 試合出場が0の年もあるので、その対策。

default_game_count = pd.DataFrame([])

for i in range(2018, last_game_year+1):
    tmp_df = pd.DataFrame([i])
    default_game_count = pd.concat([default_game_count, tmp_df], axis=0)

default_game_count.columns = ["game_year"]


# バッターとしての年間試合出場数
df_game_count_batter = pd.DataFrame(data_batter[["game_year", "game_date"]].groupby("game_year").game_date.nunique())
df_game_count_batter.reset_index(inplace=True)
df_game_count_batter.columns = ["game_year", "game_count"]
df_game_count_batter = pd.merge(default_game_count, df_game_count_batter, on="game_year", how = "left")
df_game_count_batter.fillna(0, inplace=True)
df_game_count_batter.game_count = df_game_count_batter.game_count.astype(int)

# ピッチャーとしての年間試合出場数。
df_game_count_pitcher = pd.DataFrame(data_pitcher[["game_year", "game_date"]].groupby("game_year").game_date.nunique())
df_game_count_pitcher.reset_index(inplace=True)
df_game_count_pitcher.columns = ["game_year", "game_count"]
df_game_count_pitcher = pd.merge(default_game_count, df_game_count_pitcher, on="game_year", how = "left")
df_game_count_pitcher.fillna(0, inplace=True)
df_game_count_pitcher.game_count = df_game_count_pitcher.game_count.astype(int)


# 二刀流としての試合出場数
tmp_df = pd.merge(data_batter, data_pitcher, on = ["game_year", "game_date"], how = "inner")
df_game_count_pitcher_and_batter = pd.DataFrame(tmp_df[["game_year", "game_date"]].groupby("game_year").game_date.nunique())
df_game_count_pitcher_and_batter.reset_index(inplace=True)
df_game_count_pitcher_and_batter.columns = ["game_year", "game_count"]
df_game_count_pitcher_and_batter = pd.merge(default_game_count, df_game_count_pitcher_and_batter, on="game_year", how = "left")
df_game_count_pitcher_and_batter.fillna(0, inplace=True)
df_game_count_pitcher_and_batter.game_count = df_game_count_pitcher_and_batter.game_count.astype(int)


# バッターまたはピッチャーとしての年間試合出場数
tmp_df = pd.merge(data_batter, data_pitcher, on = ["game_year", "game_date"], how = "outer")
df_game_count_pitcher_or_batter = pd.DataFrame(tmp_df[["game_year", "game_date"]].groupby("game_year").game_date.nunique())
df_game_count_pitcher_or_batter.reset_index(inplace=True)
df_game_count_pitcher_or_batter.columns = ["game_year", "game_count"]
df_game_count_pitcher_or_batter = pd.merge(default_game_count, df_game_count_pitcher_or_batter, on="game_year", how = "left")
df_game_count_pitcher_or_batter.fillna(0, inplace=True)
df_game_count_pitcher_or_batter.game_count = df_game_count_pitcher_or_batter.game_count.astype(int)

show_many_dfs(df_game_count_pitcher_and_batter, df_game_count_pitcher, df_game_count_batter, df_game_count_pitcher_or_batter)

,game_year,game_count
0,2018,0
1,2019,0
2,2020,0
3,2021,20
4,2022,28
5,2023,23
,game_year,game_count
0,2018,10
1,2019,0
2,2020,2


##### <font color="plum">レイアウトの作成</font>


In [13]:
# MLB試合出場数のレイアウト
layout_game_count= dbc.Container(
    [
        dbc.Row(
            [
                html.H1("MLBでの年間別試合出場数",
                       style={"margin-bottom" : "30px"})
            ],
            class_name="text-light"
        ),
        dbc.Row(
            [
                dcc.Checklist(
                    id="checkbox_game_count",
                    options=[
                        {"label": "ピッチャー&バッター", "value": "pitcher_and_batter"},
                        {"label": "ピッチャー", "value": "pitcher"},
                        {"label": "バッター", "value": "batter"},
                        {"label": "ピッチャーorバッター", "value": "pitcher_or_batter"}
                    ],
                    value=["pitcher_and_batter", "pitcher", "batter", "pitcher_or_batter"],
                )
            ],
            class_name="text-light"
        ),
        dbc.Row(style={"margin-top": "20px"}),
        
        dbc.Row(
            [
                dbc.Col(
                    dcc.Graph(id="年間試合出場数",
                              style={"height": "70vh",
                                     "border": "1px solid black"},
                    ),
                    width=11,
                    className="mx-auto"
                )
            ],
            className='bg-dark'
        )
    ],
    # fluid=True
)
color_mapping = {
    "pitcher_and_batter": "violet",
    "pitcher": "seagreen",
    "batter": "royalblue",
    "pitcher_or_batter": "orangered"
}
@app.callback(
    Output("年間試合出場数", "figure"),
    Input("checkbox_game_count", "value")
)
def update_graph(selected_checkboxes):
    data = []
    game_year = df_game_count_batter["game_year"]  # X軸のデータ

    # 選択された順序に応じてデータを追加（チェックボックスで選択し直しても、順番は変わらない）
    for checkbox in ["pitcher_and_batter", "pitcher", "batter", "pitcher_or_batter"]:
        if checkbox in selected_checkboxes:
            color = color_mapping.get(checkbox)

            if checkbox == "pitcher_and_batter":
                name = "ピッチャー&バッター"
                game_count = df_game_count_pitcher_and_batter["game_count"]
            elif checkbox == "pitcher":
                name = "ピッチャー"
                game_count = df_game_count_pitcher["game_count"]
            elif checkbox == "batter":
                name = "バッター"
                game_count = df_game_count_batter["game_count"]
            elif checkbox == "pitcher_or_batter":
                name = "ピッチャーorバッター"
                game_count = df_game_count_pitcher_or_batter["game_count"]

            data.append(go.Bar(x=game_year,
                               y=game_count,
                               name=name,
                               marker=dict(color=color),
                               hovertemplate="試合出場回数：%{y}回"
                               ))

    return {
        "data": data,
        "layout": go.Layout(
            barmode="group",
            bargroupgap=0.1,
            xaxis_title="年",
            yaxis_title="試合出場回数（回）",
            xaxis_title_font={"size": 20},
            yaxis_title_font={"size": 20}
        )
    }


### <font color="turquoise">バッティングデータのデータフレーム作成・レイアウト作成</font>

#### <font color="springgreen">年間別ホームラン数</font>

##### <font color="plum">データフレームの作成</font>

In [14]:
# ホームラン数のデータフレームを作成
df_HR_Ohtani = pd.DataFrame(data_batter.query("events == 'home_run'").groupby("game_year").events.count())
df_HR_Ohtani.reset_index(inplace=True)
df_HR_Ohtani.columns = ["game_year", "count"]
df_HR_Ohtani["player_name"] = "大谷翔平"

df_homerun_king = pd.read_csv("./data/homerun_king.csv")
df_homerun_king.columns = df_HR_Ohtani.columns

show_many_dfs(df_HR_Ohtani, df_homerun_king)

,game_year,count,player_name
0,2018,22,大谷翔平
1,2019,18,大谷翔平
2,2020,7,大谷翔平
3,2021,46,大谷翔平
4,2022,34,大谷翔平
5,2023,44,大谷翔平
,game_year,count,player_name
0,2018,48,クリス・デービス
1,2019,48,ホルヘ・ソレア
2,2020,22,ルーク・ボイド


##### <font color="plum">レイアウトの作成</font>


In [15]:
x_axis_Ohtani = df_HR_Ohtani["game_year"]
y_axis_Ohtani = df_HR_Ohtani["count"]

x_axis_homerun_king = df_homerun_king["game_year"]
y_axis_homerun_king = df_homerun_king["count"]

# ホームラン王のラベルを設定
labels_homerun_king = df_homerun_king["player_name"].to_list()
# ラベルが長すぎると文字が小さくなってしまうので、省略したラベルを作成する。
# 11文字以上の場合は、先頭5文字+"..."とする。
short_labels = [label[:5] + "..." if len(label) > 10 else label for label in labels_homerun_king]

layout_batting_homerun_count = dbc.Container(
    [
        dbc.Row(
            [
                html.H1('年間別ホームラン数',
                       className="text-light"),
                dbc.Col(
                    dcc.Graph(
                        figure={
                            "data": [
                                        go.Bar(x=x_axis_Ohtani,
                                                y=y_axis_Ohtani,
                                                name="大谷翔平",
                                                marker=dict(color="orangered"),
                                                hovertemplate='選手名：大谷翔平<br>ホームラン数： %{y}本'
                                                ),
                                         go.Bar(x=x_axis_homerun_king,
                                                y=y_axis_homerun_king,
                                                name="ホームラン王",
                                                marker=dict(color="darkturquoise"),
                                                text=short_labels,
                                                textposition="outside",
                                                textfont=dict(size=8),
                                                hovertext=labels_homerun_king,
                                                hovertemplate='選手名：%{hovertext}<br>ホームラン数： %{y}本',
                                                )
                                    ],
                            "layout": go.Layout(
                                                barmode="group",
                                                bargroupgap=0.1,
                                                xaxis_title="年",
                                                yaxis_title="ホームラン数",
                                                xaxis_title_font={"size": 20},
                                                yaxis_title_font={"size": 20}
                                        )
                        },
                        style={"height": "95%",
                               "border": "1px solid black"}
                    ),
                    width=11,
                    className="mx-auto"
                )
            ],
            style={"height": "100vh"}
        )
    ],
    # fluid=True
)

#### <font color="springgreen">投げられたコース</font>

##### <font color="plum">関数の定義</font>

In [16]:
def make_batting_zone_compare(batting_zone_items_id, Output_id):

    batting_zone_compare = dbc.Container(
        [
            dbc.Row(
                [   
                    dbc.Col([
                        dcc.RadioItems(
                            id=batting_zone_items_id,
                            options=[
                                {"label": "シングルヒット", "value": "single"},             # events
                                {"label": "ツーベースヒット", "value": "double"},           # events
                                {"label": "スリーベースヒット", "value": "triple"},         # events
                                {"label": "ホームラン", "value": "home_run"},               # events
                                {"label": "デッドボール", "value": "hit_by_pitch"},         # events
                                {"label": "犠牲フライ", "value": "sac_fly"},                # events
                                {"label": "ファール", "value": "foul"},                     # description
                                {"label": "フィールドアウト", "value": "field_out"},        # events
                                {"label": "空振り", "value": "swinging_strike"}            # description
                            ],
                            value="single",
                            style={"columnCount":2}
                        ),
                    ])
                ],
                className="bg-primary text-white mb-4"
            ),
            dcc.Graph(id=Output_id,
                      style={"height": "100%"}
            )
        ],
        className="bg-light"
    )
    return batting_zone_compare

##### <font color="plum">レイアウトの作成</font>

In [17]:
batting_zone_compare1 = make_batting_zone_compare(batting_zone_items_id="batting_zone_items_id_1",
                                                  Output_id="batting_zone_compare1")

batting_zone_compare2 = make_batting_zone_compare(batting_zone_items_id="batting_zone_items_id_2",
                                                  Output_id="batting_zone_compare2")

layout_batting_zone = dbc.Container(
    [
        dbc.Row(
            [
                html.H1("投げられたコース",
                       className="text-light"),
                
                dbc.Row(style={"margin-top": "30px"}),

                dbc.Row(
                    [
                        dbc.Col(batting_zone_compare1, width=6),
                        dbc.Col(batting_zone_compare2, width=6)
                    ]
                ),
                html.Small("・キャッチャー・球審目線です。",
                           className="text-light"),
                html.Small("・大谷翔平選手は左打ちなので、ヒートマップ右側の打席に立ちます。",
                           className="text-light")
            ]
        )
    ],
    # fluid=True
)
@app.callback(
    Output("batting_zone_compare1", "figure"),
    Output("batting_zone_compare2", "figure"),
    Input("batting_zone_items_id_1", "value"),
    Input("batting_zone_items_id_2", "value")
)
def update_graph(selected_items_1, selected_items_2):
    
    df_zone_1 = make_df_zone_count(data_batter.query("events == @selected_items_1 or description == @selected_items_1"))
    df_zone_2 = make_df_zone_count(data_batter.query("events == @selected_items_2 or description == @selected_items_2"))
    
    # フィギュアを作成
    # zminでスケールバーの下限値を0に固定。
    fig_zone_1 = px.imshow(df_zone_1,
                           color_continuous_scale=[[0, "mistyrose"],[1, "red"]],
                           zmin=0)
    fig_zone_1.update_traces(hovertemplate="%{z} 球")  # カラー値に「球」の単位を追加

    fig_zone_2 = px.imshow(df_zone_2,
                           color_continuous_scale=[[0, "mistyrose"], [1, "red"]],
                           zmin=0)
    fig_zone_2.update_traces(hovertemplate="%{z} 球")

    
    # 軸のラベルと目盛りラベルを非表示
    fig_zone_1.update_layout(xaxis_showticklabels=False, yaxis_showticklabels=False)
    fig_zone_2.update_layout(xaxis_showticklabels=False, yaxis_showticklabels=False)

    return fig_zone_1, fig_zone_2

#### <font color="springgreen">投げられた球種</font>

##### <font color="plum">関数の定義</font>

In [18]:
def make_pitched_type_compare(pitched_type_items_id, Output_id):

    batting_zone_compare = dbc.Container(
        [
            dbc.Row(
                [   
                    dbc.Col([
                        dcc.RadioItems(
                            id=pitched_type_items_id,
                            options=[
                                {"label": "シングルヒット", "value": "single"},             # events
                                {"label": "ツーベースヒット", "value": "double"},           # events
                                {"label": "スリーベースヒット", "value": "triple"},         # events
                                {"label": "ホームラン", "value": "home_run"},               # events
                                {"label": "デッドボール", "value": "hit_by_pitch"},         # events
                                {"label": "犠牲フライ", "value": "sac_fly"},                # events
                                {"label": "ファール", "value": "foul"},                     # description
                                {"label": "フィールドアウト", "value": "field_out"},        # events
                                {"label": "空振り", "value": "swinging_strike"}            # description
                            ],
                            value="single",
                            style={"columnCount":2}
                        )
                    ])
                ],
                className="bg-primary text-white mb-4"
            ),
            dcc.Graph(id=Output_id,
                      style={"height": "90vh"}
            )
        ],
        className="bg-light"
    )
    return batting_zone_compare

##### <font color="plum">レイアウトの作成</font>

In [19]:
pitched_type_compare1 = make_pitched_type_compare(pitched_type_items_id="pitched_type_items_id_1",
                                                  Output_id="pitched_type_compare1")
pitched_type_compare2 = make_pitched_type_compare(pitched_type_items_id="pitched_type_items_id_2",
                                                  Output_id="pitched_type_compare2")

layout_pitched_type = dbc.Container(
    [
        dbc.Row(
            [
                html.H1("投げられた球種",
                       className="text-light"),
                
                dbc.Row(style={"margin-top": "30px"}),

                dbc.Row(
                    [
                        dbc.Col(pitched_type_compare1, width=6),
                        dbc.Col(pitched_type_compare2, width=6)
                    ]
                )
            ]
        )
    ],
    # fluid=True
)
@app.callback(
    Output("pitched_type_compare1", "figure"),
    Output("pitched_type_compare2", "figure"),
    Input("pitched_type_items_id_1", "value"),
    Input("pitched_type_items_id_2", "value")
)
def update_graph(selected_items_1, selected_items_2):
    
    df_pitched_type_1 = pd.DataFrame(data_batter.query("events == @selected_items_1 or description == @selected_items_1")
                                     .pitch_name.value_counts())
    df_pitched_type_1.reset_index(inplace=True)
    df_pitched_type_1.columns=["pitch_name", "count"]
    df_pitched_type_1.sort_values(by="count")
        
    df_pitched_type_2 = pd.DataFrame(data_batter.query("events == @selected_items_2 or description == @selected_items_2")
                                     .pitch_name.value_counts())
    df_pitched_type_2.reset_index(inplace=True)
    df_pitched_type_2.columns=["pitch_name", "count"]
    df_pitched_type_2.sort_values(by="count")
    
    # # ラベルごとに異なる色を指定
    colors = px.colors.qualitative.Plotly[:len(data_pitcher.pitch_name.unique())]
    
    # フィギュアを作成
    fig_pitched_type_1 ={"data":[go.Pie(labels=df_pitched_type_1["pitch_name"],
                                        values=df_pitched_type_1["count"],
                                        direction="clockwise",
                                        hovertemplate="%{label} <br> %{value} 球 <br> (%{percent})",
                                        textinfo="percent+label+value",
                                        marker=dict(colors=colors)
                                        )]}
    
    fig_pitched_type_2 ={"data":[go.Pie(labels=df_pitched_type_2["pitch_name"],
                                        values=df_pitched_type_2["count"],
                                        direction="clockwise",
                                        hovertemplate="%{label} <br> %{value} 球 <br> (%{percent})",
                                        textinfo="percent+label+value",
                                        marker=dict(colors=colors)
                                        )]}
    


    return fig_pitched_type_1, fig_pitched_type_2

### <font color="turquoise">ピッチングデータのデータフレーム作成・レイアウト作成</font>

#### <font color="springgreen">持ち玉と配球</font>

##### <font color="plum">レイアウトの作成</font>

In [20]:
options_game_year = data_pitcher.game_year.sort_values(ascending=True).astype(object).unique()
options_game_year = np.append(options_game_year, "全て")


layout_pitch_type_ohtani_have = dbc.Container(
    [
        html.H1("持ち玉と配球",
                className="text-light"),
        dbc.Row(
            [   
                dbc.Col(
                    [   
                        dbc.Row(
                            [
                                dbc.Col(
                                    [
                                        dcc.RadioItems(
                                            id="id_stand_1",
                                            options=[{"label": "右打者", "value": "R"},
                                                     {"label": "左打者", "value": "L"},
                                                     {"label": "全て", "value": "全て"}],
                                            value="全て",
                                            style={"width": "100%"},
                                        ),
                                    ]
                                ),
                                dbc.Col(
                                    [
                                        dcc.RadioItems(
                                            id="id_game_year_1",
                                            options=options_game_year,
                                            value="全て",
                                            style={"width": "100%"}
                                        )
                                    ]
                                )
                            ],
                            style={"background-color":"orangered"}
                        )           
                    ],
                    width=3
                ),
                dbc.Col(
                    [   
                        dbc.Row(
                            [
                                dbc.Col(
                                    [
                                        dcc.RadioItems(
                                            id="id_stand_2",
                                            options=[{"label": "右打者", "value": "R"},
                                                     {"label": "左打者", "value": "L"},
                                                     {"label": "全て", "value": "全て"}],
                                            value="全て",
                                            style={"width": "100%"}
                                        ),
                                    ]
                                ),
                                dbc.Col(
                                    [
                                        dcc.RadioItems(
                                            id="id_game_year_2",
                                            options=options_game_year,
                                            value="全て",
                                            style={"width": "100%"}
                                        )
                                    ]
                                )
                            ],
                            style={"background-color":"darkturquoise"}
                        )           
                    ],
                    width=3
                ),
                dbc.Col(className="bg-dark"),
                dcc.Graph(id="bar_pitch_type",
                style={"height": "70vh"}
                )
            ],
            className='bg-light text-white mb-4 text-center'
        )
    ],
    # fluid=True
)
@app.callback(
    Output("bar_pitch_type", "figure"),
    Input("id_stand_1", "value"),
    Input("id_game_year_1", "value"),
    Input("id_stand_2", "value"),
    Input("id_game_year_2", "value")
)
def update_graph(id_stand_1, id_game_year_1, id_stand_2, id_game_year_2):
    
    def make_pitch_type_count(id_stand, id_game_year):
        if id_stand == "全て" and id_game_year == "全て":
            pitch_type = data_pitcher.pitch_name.sort_values().unique()  # X軸のデータ
            pitch_type_count = data_pitcher.pitch_name.sort_values().value_counts().values
            
        elif id_stand == "全て" and id_game_year != "全て":
            tmp_df = data_pitcher.query("game_year == @id_game_year")
            pitch_type = tmp_df.pitch_name.sort_values().unique()  # X軸のデータ
            pitch_type_count = tmp_df.pitch_name.sort_values().value_counts().values
            
        elif id_stand != "全て" and id_game_year == "全て":
            tmp_df = data_pitcher.query("stand == @id_stand")
            pitch_type = tmp_df.pitch_name.sort_values().unique()  # X軸のデータ
            pitch_type_count = tmp_df.pitch_name.sort_values().value_counts().values
        
        else:
            tmp_df = data_pitcher.query("stand == @id_stand and game_year == @id_game_year")
            pitch_type = tmp_df.pitch_name.sort_values().unique()  # X軸のデータ
            pitch_type_count = tmp_df.pitch_name.sort_values().value_counts().values
        
        if id_stand == "全て":
            stand_label = "全て"
        elif id_stand == "R":
            stand_label = "右打者"
        elif id_stand == "L":
            stand_label = "左打者"
        

        return pitch_type, pitch_type_count, stand_label
    
    pitch_type_1, pitch_type_count_1, stand_label_1 = make_pitch_type_count(id_stand_1, id_game_year_1)
    pitch_type_2, pitch_type_count_2, stand_label_2 = make_pitch_type_count(id_stand_2, id_game_year_2)
    
    figure={"data": [
                        go.Bar(
                            x=pitch_type_1,
                            y=pitch_type_count_1,
                            marker=dict(color="orangered"),
                            hovertemplate='球種：%{x}<br>投球数： %{y}球',
                            text=pitch_type_count_1,
                            textposition="outside",
                            name=f"{stand_label_1}-{id_game_year_1}"
                        ),
                        go.Bar(
                            x=pitch_type_2,
                            y=pitch_type_count_2,
                            marker=dict(color="darkturquoise"),
                            hovertemplate='球種：%{x}<br>投球数： %{y}球',
                            text=pitch_type_count_2,
                            textposition="outside",
                            name=f"{stand_label_2}-{id_game_year_2}"
                        )
                    ],
            "layout": go.Layout(
                            barmode="group",
                            bargroupgap=0.1,
                            xaxis_title="年",
                            yaxis_title="ホームラン数",
                            xaxis_title_font={"size": 20},
                            yaxis_title_font={"size": 20}
                    )
    }
    return figure

#### <font color="springgreen">球種ごとのデータ</font>

##### <font color="plum">レイアウトの作成</font>

In [21]:
options_pitch_type = data_pitcher.pitch_name.unique()

layout_pitch_type_data = dbc.Container(
    [
        dbc.Row(
            [
                html.H1("球種ごとのデータ",
                        className="text-light"),
                dbc.Row(
                    [
                        dbc.Col(
                            [
                                dcc.RadioItems(
                                    id="id_pitch_type_for_data",
                                    options=options_pitch_type,
                                    value="4-Seam Fastball",
                                    style={"columnCount":3}
                                ),
                            ],
                            width=6
                        ),
                        dbc.Col(width=3),
                        dbc.Col(
                            [
                                dcc.RadioItems(
                                    id="id_stand_for_pitch_type_data",
                                    options=[{"label": "右打者", "value": "R"},
                                             {"label": "左打者", "value": "L"},
                                             {"label": "全て", "value": "全て"}],
                                    value="全て",
                                )
                            ],
                            width=3
                        )
                    ],
                    className="bg-primary text-white"
                ),
                dbc.Col(
                    [
                        dbc.Row(
                            [
                                dcc.Graph(id="pitch_type_speed",
                                          style={"height":"40vh"})
                            ]
                        ),
                        dbc.Row(
                            [
                                dcc.Graph(id="id_zone_count",
                                          style={"height":"40vh"})
                            ],
                        )
                    ],
                    width=3
                ),
                dbc.Col(
                    [
                        dbc.Col(
                            [
                                dcc.Graph(id="id_description_count",
                                          style={"height":"80vh"})
                            ]
                        )
                    ],
                    width=9
                )
            ]
        )
    ],
    # fluid=True
)
@app.callback(
    Output("pitch_type_speed", "figure"),
    Output("id_description_count", "figure"),
    Output("id_zone_count", "figure"),
    Input("id_pitch_type_for_data", "value"),
    Input("id_stand_for_pitch_type_data", "value")
)
def update_graph(id_pitch_type_for_data, id_stand_for_pitch_type_data):
    
    if id_stand_for_pitch_type_data == "全て":
        tmp_df = data_pitcher.query("pitch_name == @id_pitch_type_for_data")
    else:
        tmp_df = data_pitcher.query("pitch_name == @id_pitch_type_for_data and stand == @id_stand_for_pitch_type_data")
    
    if id_stand_for_pitch_type_data == "全て":
        stand_label = "全て"
    elif id_stand_for_pitch_type_data == "R":
        stand_label = "右打者"
    elif id_stand_for_pitch_type_data == "L":
        stand_label = "左打者"

    # tmp_dfが0行のとき、以下のグラフ作成でコールバックエラーとなってしまうため、
    # NaNで埋め尽くされた1行を作成する。
    # これによってコールバックエラーなく、グラフが表示されなくなる。
    if len(tmp_df) == 0:
        tmp_df = data_pitcher[0:1]
        for i in tmp_df.columns:
            tmp_df.loc[0:i] = np.nan

    
    
### 球速の箱ひげ図について###
    figure_release_speed=px.box(tmp_df,
                                x="pitch_name",
                                y="release_speed",
                                title='球速')
    
    figure_release_speed.update_xaxes(title_text="球種")
    
    release_speed_min = data_pitcher.release_speed.min()-2
    release_speed_max = data_pitcher.release_speed.max()+2
    
    figure_release_speed.update_yaxes(title_text="球速  (km/h)",
                                      range=[release_speed_min, release_speed_max])

    # ホバーモードを設定
    figure_release_speed.update_layout(hovermode='x')



### 投げたコース
    df_zone = make_df_zone_count(tmp_df)
    
    # フィギュアを作成
    # zminでスケールバーの下限値を0に固定。
    figure_zone = px.imshow(df_zone,
                            color_continuous_scale=[[0, "mistyrose"], [1, "red"]],
                            zmin=0,
                            title='投げたコース')
    figure_zone.update_traces(hovertemplate="%{z} 球")  # カラー値に「球」の単位を追加

    
    # 軸のラベルと目盛りラベルを非表示
    figure_zone.update_layout(xaxis_showticklabels=False, yaxis_showticklabels=False)
    
    
### descriptionの棒グラフについて###
    """
    対右打者のスローボールが0なのが原因で挙動がおかしい。
    """
    figure_description={
        "data": [ go.Bar(x=tmp_df["description"].sort_values().unique(),
                         y=tmp_df["description"].sort_values().value_counts().values,
                         name=f"{id_pitch_type_for_data}-{stand_label}",
                         marker=dict(color="darkturquoise"),
                         hovertemplate="成績：%{x}<br>投球数： %{y}球",
                         text=tmp_df["description"].sort_values().value_counts().values,
                         textposition="outside"
                         )
                ],
        "layout": go.Layout(
                            xaxis_title="成績",
                            yaxis_title="投球数",
                            xaxis_title_font={"size": 20},
                            yaxis_title_font={"size": 20},
                            yaxis=dict(range=[0, tmp_df.description.value_counts().max()*1.2]),
                            title='成績'
                    )
    }
    
    return figure_release_speed, figure_description, figure_zone

### <font color="turquoise">「作成中」レイアウト</font>

In [22]:
# その他のレイアウト
layout_for_else = dbc.Container(
    [
        dbc.Row(
            [
                html.H1("作成中",
                        className='text-light')
            ],
            style={"height": "100vh",
                   "text-align": "center"},
            className="d-flex align-items-center"
        )
    ],
    # fluid=True
)

## <font color="tomato">画面表示の設定について</font>

In [23]:
app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(sidebar,
                        width=2,
                        class_name="bg-dark"
                ),
                dbc.Col(
                    html.Div(
                        [
                            ###################################
                            ###################################
                            #ここに作成したレイアウトを格納すること。
                            ###################################
                            ###################################
                            layout_home,
                            layout_game_count,    #html.Divに格納しないとエラーが出る。
                            batting_zone_compare1,
                            batting_zone_compare2,
                            pitched_type_compare1,
                            pitched_type_compare2,
                            layout_pitch_type_ohtani_have,
                            layout_pitch_type_data
                        ]
                    ),
                    id='出力の選択',
                    width=10,
                    class_name="bg-dark"
                )
            ],
            style={"height": "100vh"}
        )
    ],
    fluid=True
)

In [24]:
@app.callback(
    Output('出力の選択', 'children'),
    Input('表示させるデータ_セレクトボタン', 'value'),
    Input('データの種類_セレクトボタン', 'value')
)
def select_layout(selected_value1, selected_value2):
    if selected_value1 == 'ホーム':
        return layout_home
    
    elif selected_value1 == '基本データ' and selected_value2 == '基本情報':
        return layout_basic_information
    
    elif selected_value1 == '基本データ' and selected_value2 == '経歴':
        return layout_career
    
    elif selected_value1 == '基本データ' and selected_value2 == '獲得タイトル':
        return layout_title
    
    elif selected_value1 == '基本データ' and selected_value2 == 'その他の逸話':
        return layout_others
    
    elif selected_value1 == 'MLB試合出場数':
        return layout_game_count
    
    elif selected_value1 == 'バッティングデータ' and selected_value2 == '年間別ホームラン数':
        return layout_batting_homerun_count
    
    elif selected_value1 == 'バッティングデータ' and selected_value2 == '投げられたコース':
        return layout_batting_zone
    
    elif selected_value1 == 'バッティングデータ' and selected_value2 == '投げられた球種':
        return layout_pitched_type
    
    elif selected_value1 == 'ピッチングデータ' and selected_value2 == '持ち玉と配球':
        return layout_pitch_type_ohtani_have
    
    elif selected_value1 == 'ピッチングデータ' and selected_value2 == '球種ごとのデータ':
        return layout_pitch_type_data
    
    else:
        return layout_for_else

## <font color="tomato">サーバーの実行</font>

In [25]:
if __name__ == "__main__":
    app.run_server(debug=True, port=8050)